In [6]:
import requests
import pandas as pd

In [7]:
data_url='https://wapi.gogoro.com/tw/api/vm/list'
r = requests.get(data_url,stream = True) #stream=True #開啟時時續續的下載的方式
r.encoding = 'utf-8'
print(r.encoding)
with open('./data.txt','wb') as f: 
    for chunk in r.iter_content(chunk_size=32): #chunk_size #設定每次下載檔案的大小
        f.write(chunk)  #每一次迴圈儲存一次下載下來的內容
f.close

utf-8


<function BufferedWriter.close>

In [28]:
import json
import pandas as pd
with open('./new_data.json', 'r') as f:
    data_json = json.load(f)
#remove unused element [result, message, error_code]
data_json.pop('result')
data_json.pop('message')
data_json.pop('error_code')
# for element in data_json["data"]:
#     print(element)
with open('final_data.json', 'w') as outfile:
    json.dump(data_json, outfile)
df = pd.DataFrame(list(data_json['data']))
df = df.set_index('Id')
print(df)


                                                                                LocName  \
Id                                                                                        
865a6d73-ab7d-4176-a542-09898fbf7dca  { List : [{ Value :  Gogoro Huwei Guangfu RS C...   
cc480eeb-93c4-49b1-9b12-15474249b5bf  { List : [{ Value :  Pxmart YL Douliu Store , ...   
5a72a2fb-505c-476b-a796-17fcf769c99a  { List : [{ Value :  7-ELEVEN Zhu Danwan Store...   
41c44a19-d494-4845-a659-19d5cba5b29a  { List : [{ Value :  CPC Tuku GS , Lang :  en-...   
5dfa7fd8-2148-4fb2-b3b3-2d98949c0227  { List : [{ Value :  Pxmart HW Zhongzhen Store...   
...                                                                                 ...   
b40c5eb7-7606-4b93-87ca-02fe58da5838  { List :[{ Value : Familymart Yongkang Zhongxi...   
f3a88903-7cc3-482c-856f-02a23ca5b87b  { List :[{ Value : Fuqianting 18 Parking Store...   
16a11557-2db4-4fb9-a6d0-06bde48315fd  { List :[{ Value : CPC Zhonghualudong GS , Lan...   